In [ ]:
import numpy as np
# np.random.seed(0)
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from scipy.misc import imread
from scipy.misc import imresize

import pickle
import matplotlib.image as mpimg
from scipy.misc import imread
from scipy.misc import imresize

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rc('figure', figsize=(10, 10))

In [ ]:
gt = pickle.load(open('ground-truth.pkl', 'rb'))
gt['object-detection-crowdai/1479498371963069978.jpg']

In [ ]:
gt['object-detection-crowdai/1479498371963069978.jpg'][:, 0]

In [ ]:
image = mpimg.imread('resources/udacity-dataset/object-detection-crowdai/1479498371963069978.jpg')
h, w, _ = image.shape
image.shape

In [ ]:
plt.imshow(image)

In [ ]:
def draw_each(img, bboxes):
    for bbox in bboxes:
        xmin = int(round(bbox[0] * img.shape[1]))
        ymin = int(round(bbox[1] * img.shape[0]))
        xmax = int(round(bbox[2] * img.shape[1]))
        ymax = int(round(bbox[3] * img.shape[0]))
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax),(0, 255, 0), 1)
    return img

def draw(org_img, org_bboxes, trans_img, trans_bboxes):
    fig = plt.figure(figsize=(30, 25))
    fig_dims = (1, 2)
    ax1 = plt.subplot2grid(fig_dims, (0, 0))
    ax1.set_title('original')
    ax1.imshow(draw_each(org_img, org_bboxes))
    
    ax2 = plt.subplot2grid(fig_dims, (0, 1))
    ax2.set_title('augmented')
    ax2.imshow(draw_each(trans_img, trans_bboxes))
    plt.show()

In [ ]:
def denormalize(img, annotations):
    h, w, _ = img.shape
    annotations = np.copy(annotations)
    annotations[:, 0] = annotations[:, 0] * w
    annotations[:, 2] = annotations[:, 2] * w
    annotations[:, 1] = annotations[:, 1] * h
    annotations[:, 3] = annotations[:, 3] * h
    return annotations

def normalize(img, annotations):
    h, w, _ = img.shape
    annotations = np.copy(annotations)
    annotations[:, 0] = annotations[:, 0] / w
    annotations[:, 2] = annotations[:, 2] / w
    annotations[:, 1] = annotations[:, 1] / h
    annotations[:, 3] = annotations[:, 3] / h
    return annotations

## Image Translation

In [ ]:
def translate(img, annotations, trans_range=20):
    # Translation augmentation
    annotations = np.copy(annotations)
    h, w, _ = img.shape
    tr_x = trans_range * np.random.uniform() - trans_range/2
    tr_y = trans_range * np.random.uniform() - trans_range/2

    Trans_M = np.float32([[1, 0, tr_x], [0, 1, tr_y]])
    
    annotations[:, 0] = annotations[:, 0] + tr_x # xmin
    annotations[:, 2] = annotations[:, 2] + tr_x # xmax
    annotations[:, 1] = annotations[:, 1] + tr_y # ymin
    annotations[:, 3] = annotations[:, 3] + tr_y # ymax
    
    img_translated = cv2.warpAffine(img, Trans_M, (w, h))

    return img_translated, annotations

image = mpimg.imread('resources/udacity-dataset/object-detection-crowdai/1479498371963069978.jpg')
annotations = gt['object-detection-crowdai/1479498371963069978.jpg']
img_tr, bboxes_tr = translate(image, denormalize(image, annotations), trans_range=200)
bboxes_tr = normalize(img_tr, bboxes_tr)
draw(image, annotations, img_tr, bboxes_tr)

## Image Stretching

In [ ]:
def stretch(img, annotations, scale_range=50):
    # Stretching augmentation 
    annotations = np.copy(annotations)
    tr_x1 = scale_range * np.random.uniform()
    tr_y1 = scale_range * np.random.uniform()
    p1 = (tr_x1, tr_y1)
    tr_x2 = scale_range * np.random.uniform()
    tr_y2 = scale_range * np.random.uniform()
    p2 = (img.shape[1] - tr_x2, tr_y1)

    p3 = (img.shape[1] - tr_x2, img.shape[0] - tr_y2)
    p4 = (tr_x1, img.shape[0] - tr_y2)

    pts1 = np.float32([[p1[0], p1[1]], 
                       [p2[0], p2[1]],
                       [p3[0], p3[1]], 
                       [p4[0], p4[1]]])
    pts2 = np.float32([[0, 0],
                       [img.shape[1], 0], 
                       [img.shape[1], img.shape[0]],
                       [0, img.shape[0]]])

    M = cv2.getPerspectiveTransform(pts1, pts2)
    img = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))
    img = np.array(img, dtype=np.uint8)
    
    annotations[:, 0] = (annotations[:, 0] - p1[0]) / (p2[0] - p1[0]) * img.shape[1] # xmin
    annotations[:, 2] = (annotations[:, 2] - p1[0]) / (p2[0] - p1[0]) * img.shape[1] # xmax
    annotations[:, 1] = (annotations[:, 1] - p1[1]) / (p3[1] - p1[1]) * img.shape[0] # ymin
    annotations[:, 3] = (annotations[:, 3] - p1[1]) / (p3[1] - p1[1]) * img.shape[0] # ymax
    
    return img, annotations

image = mpimg.imread('resources/udacity-dataset/object-detection-crowdai/1479498371963069978.jpg')
annotations = gt['object-detection-crowdai/1479498371963069978.jpg']
img_scaled, annotations_scaled = stretch(image, denormalize(image, annotations), 10)
annotations_scaled = normalize(img_scaled, annotations_scaled)

draw(image, annotations, img_scaled, annotations_scaled)

## Horizontal Flip

In [ ]:
def horizontal_flip(img, y):
    img = img[:, ::-1]
    y[:, [0, 2]] = 1 - y[:, [2, 0]]
    return (img, y)

image = mpimg.imread('resources/udacity-dataset/object-detection-crowdai/1479498371963069978.jpg')
annotations = gt['object-detection-crowdai/1479498371963069978.jpg']
image_flipped, annotations_flipped = horizontal_flip(image, np.copy(annotations))
draw(image, annotations, np.array(image_flipped), annotations_flipped)

## Brightness

In [ ]:
def brightness(image):
#     alpha = randint(1, 5) 
#     beta = randint(-100, 100) 
#     result = cv2.addWeighted(rgb, alpha, np.zeros(rgb.shape, rgb.dtype), 0, beta) 
#     return np.clip(result, 0, 255)
    img = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    img = np.array(img, dtype = np.float64)
    random_bright = .5 + np.random.uniform()
    img[:, :, 2] = img[:, :, 2] * random_bright
    img[:, :, 2][img[:, :, 2] > 255]  = 255
    img = np.array(img, dtype = np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    return img

image = mpimg.imread('resources/udacity-dataset/object-detection-crowdai/1479498371963069978.jpg')
annotations = gt['object-detection-crowdai/1479498371963069978.jpg']
image_b = brightness(np.copy(image))
annotations_b = np.copy(annotations)
draw(image, annotations, image_b, annotations_b)

## Combination

In [ ]:
image = mpimg.imread('resources/udacity-dataset/object-detection-crowdai/1479498371963069978.jpg')
annotations = gt['object-detection-crowdai/1479498371963069978.jpg']
image_augmented, annotations_augmented = translate(image, denormalize(image, annotations), trans_range=100)
image_augmented, annotations_augmented = stretch(image_augmented, annotations_augmented, 100)
annotations_augmented = normalize(image_augmented, annotations_augmented)
image_augmented, annotations_augmented = horizontal_flip(image_augmented, annotations_augmented)

draw(image, annotations, np.array(image_augmented), annotations_augmented)

In [ ]:
image = mpimg.imread('resources/udacity-dataset/object-detection-crowdai/1479498397956718105.jpg')
annotations = gt['object-detection-crowdai/1479498397956718105.jpg']
image_augmented, annotations_augmented = translate(image, denormalize(image, annotations), trans_range=200)
image_augmented, annotations_augmented = stretch(image_augmented, annotations_augmented, 200)
annotations_augmented = normalize(image_augmented, annotations_augmented)
image_augmented, annotations_augmented = horizontal_flip(image_augmented, annotations_augmented)

draw(image, annotations, np.array(image_augmented), annotations_augmented)

In [ ]:
image = mpimg.imread('resources/udacity-dataset/object-detection-crowdai/1479502217225257061.jpg')
annotations = gt['object-detection-crowdai/1479502217225257061.jpg']
image_augmented, annotations_augmented = translate(image, denormalize(image, annotations), trans_range=200)
image_augmented, annotations_augmented = stretch(image_augmented, annotations_augmented, 200)
annotations_augmented = normalize(image_augmented, annotations_augmented)
image_augmented, annotations_augmented = horizontal_flip(image_augmented, annotations_augmented)

draw(image, annotations, np.array(image_augmented), annotations_augmented)

In [ ]:
image = mpimg.imread('resources/udacity-dataset/object-detection-crowdai/1479503036282378933.jpg')
annotations = gt['object-detection-crowdai/1479503036282378933.jpg']
image_augmented, annotations_augmented = translate(image, denormalize(image, annotations), trans_range=200)
image_augmented, annotations_augmented = stretch(image_augmented, annotations_augmented, 200)
annotations_augmented = normalize(image_augmented, annotations_augmented)
image_augmented, annotations_augmented = horizontal_flip(image_augmented, annotations_augmented)

draw(image, annotations, np.array(image_augmented), annotations_augmented)

In [ ]:
image = mpimg.imread('resources/udacity-dataset/object-dataset/1478019959681353555.jpg')
annotations = gt['object-dataset/1478019959681353555.jpg']
image_augmented, annotations_augmented = translate(image, denormalize(image, annotations), trans_range=200)
image_augmented, annotations_augmented = stretch(image_augmented, annotations_augmented, 200)
annotations_augmented = normalize(image_augmented, annotations_augmented)
image_augmented, annotations_augmented = horizontal_flip(image_augmented, annotations_augmented)

draw(image, annotations, np.array(image_augmented), annotations_augmented)

In [ ]:
image = imread('resources/udacity-dataset/object-dataset/1478901535246276321.jpg')
image = imresize(image, (300, 300))
annotations = gt['object-dataset/1478901535246276321.jpg']
image_augmented, annotations_augmented = translate(image, denormalize(image, annotations), trans_range=10)
image_augmented, annotations_augmented = stretch(image_augmented, annotations_augmented, 10)
annotations_augmented = normalize(image_augmented, annotations_augmented)
image_augmented, annotations_augmented = horizontal_flip(image_augmented, annotations_augmented)

draw(image, annotations, np.array(image_augmented), annotations_augmented)

In [ ]:
image = imread('resources/udacity-dataset/object-dataset/1478901532389636298.jpg')
image = imresize(image, (300, 300))
annotations = gt['object-dataset/1478901532389636298.jpg']
image_augmented, annotations_augmented = translate(image, denormalize(image, annotations), trans_range=10)
image_augmented, annotations_augmented = stretch(image_augmented, annotations_augmented, 10)
annotations_augmented = normalize(image_augmented, annotations_augmented)
image_augmented, annotations_augmented = horizontal_flip(image_augmented, annotations_augmented)

draw(image, annotations, np.array(image_augmented), annotations_augmented)